# LCEL LangChainExpressionLanguage
- 프롬프트 > LLM input > chain input > output > prompt

- Runnable 인터페이스 > 랭체인 구현하고 있다.

### SequentialCahin

In [1]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1-nano')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AE71D2F60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021AE72A48C0>, root_client=<openai.OpenAI object at 0x0000021AE5A36A20>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AE5A67CB0>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

template = PromptTemplate.from_template("{city}의 랜드마크는 어디인가요?")

chain = LLMChain(llm=llm,prompt=template)
chain

C:\Users\Admin\AppData\Local\Temp\ipykernel_18188\13390109.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm,prompt=template)


LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['city'], input_types={}, partial_variables={}, template='{city}의 랜드마크는 어디인가요?'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AE71D2F60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021AE72A48C0>, root_client=<openai.OpenAI object at 0x0000021AE5A36A20>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AE5A67CB0>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={})

In [3]:
chain.invoke({'city':'파리'})

{'city': '파리',
 'text': '파리의 대표적인 랜드마크는 에펠탑입니다. 그 외에도 노트르담 대성당, 루브르 박물관, 샹젤리제 거리, 사크레쾨르 성당, 오르세 미술관 등이 유명한 명소입니다.'}

# LCEL 파이프라인 구성

In [5]:
chain_lcel = template | llm
chain_lcel

PromptTemplate(input_variables=['city'], input_types={}, partial_variables={}, template='{city}의 랜드마크는 어디인가요?')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AE71D2F60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021AE72A48C0>, root_client=<openai.OpenAI object at 0x0000021AE5A36A20>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AE5A67CB0>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [6]:
chain_lcel.invoke({'city':'파리'})

AIMessage(content='파리의 대표적인 랜드마크로는 에펠탑이 가장 유명합니다. 이외에도 노트르담 대성당, 루브르 박물관, 샹젤리제 거리, 사크레쾨르 성당, 오르세이 미술관 등이 파리의 주요 랜드마크입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 19, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDMyqhHRrLVqkzrGAier7brhR2gyc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--56024b71-636b-4739-975a-47024d66d9cc-0', usage_metadata={'input_tokens': 19, 'output_tokens': 75, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# Sequetial chain의 구성
from langchain.chains import SequentialChain

prompt1 = PromptTemplate.from_template('{city}의 랜드마크 1개만 알려주세요. 설명은 필요없고 이름만 1개 대답해주세요')
prompt2 = PromptTemplate.from_template('{landmark}에 {transport}로 가는 방법이 무엇인가요')

chain1 = LLMChain(llm=llm, prompt=prompt1, output_key='landmark', verbose=True)
chain2 = LLMChain(llm=llm, prompt=prompt2)
seq_chain = SequentialChain(chains=[chain1,chain2], input_variables=['city', 'transport'])
seq_chain

SequentialChain(verbose=False, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['city'], input_types={}, partial_variables={}, template='{city}의 랜드마크 1개만 알려주세요. 설명은 필요없고 이름만 1개 대답해주세요'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AE71D2F60>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021AE72A48C0>, root_client=<openai.OpenAI object at 0x0000021AE5A36A20>, root_async_client=<openai.AsyncOpenAI object at 0x0000021AE5A67CB0>, model_name='gpt-4.1-nano', model_kwargs={}, openai_api_key=SecretStr('**********')), output_key='landmark', output_parser=StrOutputParser(), llm_kwargs={}), LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['landmark', 'transport'], input_types={}, partial_variables={}, template='{landmark}에 {transport}로 가는 방법이 무엇인가요'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021AE71D2F60>

In [9]:
seq_chain.invoke({'city':'서울','transport':'버스'})



> Entering new LLMChain chain...
Prompt after formatting:
서울의 랜드마크 1개만 알려주세요. 설명은 필요없고 이름만 1개 대답해주세요

> Finished chain.


{'city': '서울',
 'transport': '버스',
 'text': '남산타워(남산서울타워)에 버스로 가는 방법은 다음과 같습니다:\n\n1. **가장 일반적인 버스 노선**:\n   - **03번 버스**: 남산도서관 앞으로 오는 노선으로, 남산타워에 가까운 남산도서관 또는 남산팔각정에서 하차하실 수 있습니다.\n   - **402번 버스**: 이 버스도 남산 주변을 경유하며, 남산팔각정이나 남산도서관 정류장에서 하차할 수 있습니다.\n   - **0212번 버스**: 남산 주변 주요 정류장들을 거쳐 남산타워 근처인 ‘남산도서관’ 또는 ‘남산공원입구’ 정류장에 내릴 수 있습니다.\n\n2. **버스 환승 정보**:\n   - 서울 시내 주요 환승역(예: 충무로역, 명동역)에서 해당 버스로 갈아타시면 됩니다.\n   - 버스 정보와 노선 시간표는 서울시 버스 안내 시스템 또는 모바일 앱(네이버 지도, KakaoMap 등)을 활용하시면 더욱 정확하게 확인하실 수 있습니다.\n\n3. **버스 타는 곳**:\n   - 흔히 이용하는 출발지는 서울 도심에서 충무로역, 명동, 남대문시장 등입니다.\n   - 버스정류장에서 ‘남산공원입구’, ‘남산도서관’ 정류장에서 하차하세요.\n\n**참고**: 남산타워 주변에는 케이블카를 이용하는 것도 인기 있으며, 버스를 타고 남산케이블카 하차장 근처 또는 도보로 이동하실 수 있습니다.\n\n**도움이 필요하시면**, 출발지와 시간을 알려주시면 자세한 버스 노선 정보를 안내해드릴 수 있습니다!'}